# 第10章 泛型算法

- 标准库容器定义的操作集合惊人的小，标准库并未给每个容器添加大量功能，而是提供了一组算法，这些算法中的大多数都独立于任何特定的容器，这些算法是通用的（generic，或称泛型的）：它们可用于不同类型的容器和不同类型的元素

## 10.1 概述

- 大多数算法都定义在头文件algorithm中，标准库还在头文件numeric中定义了一组数值泛型算法
- 一般情况下，这些算法并不直接操作容器，而是遍历由两个迭代器指定的一个元素范围来进行操作，通常情况下，算法遍历范围，对其中每个元素进行一些处理
- 泛型算法本身不会执行容器的操作，它们只会运行于迭代器之上，执行迭代器的操作，这也带来了一个令人惊讶但非常必要的编程假定：算法永远不会改变底层容器的大小

## 10.2 初始泛型算法

- 除了少数例外，标准库算法都对一个范围内的元素进行操作，我们将此元素范围称为“输入范围”，接受输入范围的算法总是使用前两个参数来表示此范围，两个参数分别是指向要处理的第一个元素和尾元素之后位置的迭代器
- 虽然大多数算法遍历输入范围的方式相似，但它们使用范围中元素的方式不同，理解算法的最基本方法就是了解它们是否读取元素、改变元素或是重排元素

### 10.2.1 只读算法

- 一些算法只会读取其范围内的元素，而不改变元素，例如find

### 10.2.2 写容器元素的算法

- 一些算法将新值赋予序列中的元素，例如fill

### 10.2.3 重排容器元素的算法

- 某些算法会重排容器中元素的顺序，例如sort

## 10.3 定制操作

- 很多算法都会比较输入序列中的元素，默认情况下，这类算法使用元素类型的`<`或`==`运算符完成比较，标准库还为这些算法定义了额外的版本，允许我们提供自己定义的操作来代替默认运算符

### 10.3.1 向算法传递函数

- 如果希望在调用sort的同时采用不同的比较方式，可以使用sort的第二个版本，此版本是重载过的，它接受第三个参数，此参数是一个谓词（predicate）
- 谓词是一个可调用的表达式，其返回结果是一个能用作条件的值；标准库算法所使用的谓词分为两类：一元谓词（unary predicate，意味着它们只接受单一参数）和二元谓词（binary predicate，意味着它们有两个参数）；接受谓词参数的算法对序列中的元素调用谓词

### 10.3.2 lambda表达式

#### 介绍lambda

- 我们可以向一个算法传递任何类别的可调用对象（callable object）
- 到目前为止，我们使用过的仅有的两种可调用对象是函数和函数指针，还有其他两种可调用的对象：重载了函数调用运算符的类，以及lambda表达式
- 一个lambda表达式表示一个可调用的代码单元，可以将其理解为一个未命名的内联函数；与任何函数类似，一个lambda具有一个返回类型、一个参数列表和一个函数体，但与函数不同，lambda可能定义在函数内部，一个lambda表达式具有如下形式：
```c++
[capture list] (parameter list) -> return type {function body}
```
- 其中，capture list（捕获列表）是一个lambda所在函数中定义局部变量的列表（通常为空）；return type、parameter list和function body与任何普通函数一样，分别表示返回类型、参数列表和函数体
- 我们可以忽略参数列表和返回类型，但必须永远包含捕获列表和函数体
```c++
auto f = [] {return 42;}; // 忽略括号和参数列表等价于指定一个空参数列表；忽略返回类型，将根据函数体中的代码推断出返回类型
cout << f() << endl; // 打印42
```

#### 向lambda传递参数

```c++
[] (const string &a, const string &b) {return a.size() < b.size();}
```

#### 使用捕获列表

- 虽然一个lambda可以出现在一个函数中，使用其局部变量，但它只能使用那些明确指明的变量；一个lambda通过将局部变量包含在其捕获列表中来指出将会使用这些变量，捕获列表指引lambda在其内部包含访问局部变量所需的信息
```
[sz] (const string &a) {return s.size() >= sz;};
```

### 10.3.3 lambda捕获和返回

- 当定义一个lambda时，编译器生成一个与lambda对应的新的（未命名的）类类型
- 当向一个函数传递一个lambda时，同时定义了一个新类型和该类型的一个对象
- 默认情况下，从lambda生成的类都包含一个对应该lambda所捕获的变量的数据成员，类似任何普通类的数据成员，lambda的数据成员也在lambda对象创建时被初始化

#### 值捕获

- 类似参数传递，变量的捕获方式也可以是值或引用，到目前为止，我们的lambda采用值捕获的方式，与传值参数类似，采用值捕获的前提是变量可以拷贝，与参数不同，被捕获的变量的值是在lambda创建时拷贝，而不是调用时拷贝
```c++
void fcn1()
{
    size_t v1 = 42;
    auto f = [v1] {return v1;};
    v1 = 0;
    auto j = f(); // j为42；f保存了我们创建它时v1的拷贝
}
```

#### 引用捕获

```c++
void fcn1()
{
    size_t v1 = 42;
    auto f2 = [&v1] {return v1;};
    v1 = 0;
    auto j = f2();  // j为0；f2保存v1的引用，而非拷贝
}
```

- 当以引用方式捕获一个变量时，必须保证在lambda执行时变量是存在的

#### 隐式捕获

- 除了显式列出我们希望使用的来自所在函数的变量之外，还可以让编译器根据lambda体中的代码来推断我们要使用哪些变量；为了指示编译器推断捕获列表，应在捕获列表中写一个&或=，&告诉编译器采用捕获引用方式，=则表示采用值捕获方式

### 10.3.4 参数绑定

- bind函数可以看作一个通用的函数适配器，它接受一个可调用对象，生成一个新的可调用对象来“适应”原对象的参数列表；调用bind的一般形式为`auto newCallable = bind(callable, arg_list)`，其中，newCallable本身是一个可调用对象，arg_list是一个逗号分隔的参数列表，对应给定的callable的参数